In [1]:
# Toy LFSR lab: simulate two small LFSRs, combine them with XOR to make a keystream,
# then recover the initial states by solving a linear system over GF(2).
# This is educational only (tiny sizes).

import numpy as np
from itertools import product

# --------- Helper: GF(2) Gaussian elimination ----------
def gf2_solve(A, b):
    """
    Solve A x = b over GF(2) using Gaussian elimination.
    A: (m, n) numpy array of {0,1}
    b: (m,) numpy array of {0,1}
    Returns: x (n,) in {0,1} if a solution exists, else None.
    """
    A = A.copy() % 2
    b = b.copy() % 2
    m, n = A.shape
    row = 0
    pivot_cols = []
    for col in range(n):
        if row >= m:
            break
        # find pivot
        sel = None
        for r in range(row, m):
            if A[r, col] == 1:
                sel = r
                break
        if sel is None:
            continue
        # swap rows
        if sel != row:
            A[[row, sel]] = A[[sel, row]]
            b[[row, sel]] = b[[sel, row]]
        pivot_cols.append(col)
        # eliminate below
        for r in range(row+1, m):
            if A[r, col] == 1:
                A[r] ^= A[row]
                b[r] ^= b[row]
        row += 1

    # Check for inconsistency
    for r in range(row, m):
        if b[r] == 1 and not A[r].any():
            return None  # no solution

    # Back substitution (set free vars to 0)
    x = np.zeros(n, dtype=np.uint8)
    for r in range(row-1, -1, -1):
        # find pivot col
        piv_col = None
        for c in range(n):
            if A[r, c] == 1:
                piv_col = c
                break
        if piv_col is None:
            continue
        s = b[r]
        # subtract known x * A[r, c] for c>piv_col
        for c in range(piv_col+1, n):
            if A[r, c] == 1:
                s ^= x[c]
        x[piv_col] = s
    return x

# --------- LFSR class ----------
class LFSR:
    def __init__(self, size, taps, state=None):
        """
        size: number of bits in register
        taps: list of tap indices (0..size-1). We compute newbit = XOR(state[tap] for tap in taps)
              Convention: state[0] is the leftmost (oldest) bit; output bit will be state[-1] (rightmost)
        state: initial state as list/array of 0/1, length=size. If None, randomized.
        """
        self.size = size
        self.taps = list(taps)
        if state is None:
            rng = np.random.default_rng()
            # avoid zero state for LFSR (period 0)
            s = rng.integers(0, 2, size=size, dtype=np.uint8)
            if not s.any():
                s[0] = 1
            self.state = s
        else:
            s = np.array(state, dtype=np.uint8)
            assert s.shape == (size,)
            self.state = s.copy()

    def step(self):
        """Advance one step, return output bit (rightmost bit)."""
        out = int(self.state[-1])
        # XOR taps to form new bit
        newbit = 0
        for t in self.taps:
            newbit ^= int(self.state[t])
        # shift right and insert newbit at left
        self.state[1:] = self.state[:-1]
        self.state[0] = newbit
        return out

    def generate_bits(self, n):
        return np.array([self.step() for _ in range(n)], dtype=np.uint8)

    def copy(self):
        return LFSR(self.size, self.taps, state=self.state.copy())

    def __repr__(self):
        return f"LFSR(size={self.size}, taps={self.taps}, state={self.state.tolist()})"

# --------- Build influence matrix ----------
def influence_matrix(lfsr, out_len):
    """
    Build a matrix M (out_len x size) over GF(2) such that
    M @ s0 = outputs (mod 2), where s0 is the initial state vector (length=size).
    We do this by linearizing: for each basis vector e_i as initial state, simulate outputs.
    """
    size = lfsr.size
    M = np.zeros((out_len, size), dtype=np.uint8)
    for i in range(size):
        basis = np.zeros(size, dtype=np.uint8)
        basis[i] = 1
        tmp = LFSR(size, lfsr.taps, state=basis)
        M[:, i] = tmp.generate_bits(out_len)
    return M

# --------- Simulation parameters (small toy sizes) ----------
size1 = 5   # LFSR A size (tiny for demo)
size2 = 6   # LFSR B size

# choose taps for maximal-ish polynomials (toy choices)
taps1 = [0, 2]   # newbit = s[0] XOR s[2]
taps2 = [1, 5]   # newbit = s[1] XOR s[5]

# instantiate random initial states
rng = np.random.default_rng(12345)
s0_1 = rng.integers(0, 2, size=size1, dtype=np.uint8)
if not s0_1.any(): s0_1[0] = 1
s0_2 = rng.integers(0, 2, size=size2, dtype=np.uint8)
if not s0_2.any(): s0_2[0] = 1

l1 = LFSR(size1, taps1, state=s0_1.copy())
l2 = LFSR(size2, taps2, state=s0_2.copy())

out_len = 60  # number of output bits (must be >= size1+size2 for unique soln typically)
# generate keystream by XORing outputs of two LFSRs
bits1 = l1.generate_bits(out_len)
bits2 = l2.generate_bits(out_len)
keystream = bits1 ^ bits2

print("True initial state LFSR1:", s0_1.tolist())
print("True initial state LFSR2:", s0_2.tolist())
print("First 30 keystream bits:", "".join(map(str, keystream[:30].tolist())))

# --------- Build matrix A such that A @ [s0_1; s0_2] = keystream ---------
M1 = influence_matrix(LFSR(size1, taps1), out_len)  # out_len x size1
M2 = influence_matrix(LFSR(size2, taps2), out_len)  # out_len x size2
A = np.concatenate([M1, M2], axis=1)  # out_len x (size1+size2)
b = keystream.copy()

# Solve over GF(2)
sol = gf2_solve(A, b)
if sol is None:
    print("No solution (inconsistent). You may need more output bits.)")
else:
    rec1 = sol[:size1]
    rec2 = sol[size1:]
    print("Recovered LFSR1 initial state:", rec1.tolist())
    print("Recovered LFSR2 initial state:", rec2.tolist())
    print("Recovery correct for LFSR1?", np.array_equal(rec1, s0_1))
    print("Recovery correct for LFSR2?", np.array_equal(rec2, s0_2))

# --------- Quick sanity: test by regenerating keystream from recovered states ---------
if sol is not None:
    rl1 = LFSR(size1, taps1, state=rec1)
    rl2 = LFSR(size2, taps2, state=rec2)
    ks_check = rl1.generate_bits(out_len) ^ rl2.generate_bits(out_len)
    print("Keystream matches recovered states?", np.array_equal(ks_check, keystream))

# If you'd like, experiment with different sizes and taps above.



True initial state LFSR1: [1, 1, 1, 1, 1]
True initial state LFSR2: [1, 1, 1, 1, 0, 0]
First 30 keystream bits: 110001010100010000010101000100
Recovered LFSR1 initial state: [0, 0, 0, 1, 1]
Recovered LFSR2 initial state: [1, 0, 0, 0, 0, 0]
Recovery correct for LFSR1? False
Recovery correct for LFSR2? False
Keystream matches recovered states? True


In [2]:
# Toy brute-force example demonstrating exhaustive key search on a small Feistel-style block cipher.
# This code implements:
#  - a tiny 16-bit block Feistel cipher with 4 rounds
#  - keyspace of 16-bit (0..65535)
#  - encryption of a single known plaintext -> ciphertext
#  - brute-force search over all keys to recover the encryption key
#
# Run in this notebook; outputs show the real key and the recovered key.
import random
from time import perf_counter

BLOCK_BITS = 16
HALF_BITS = BLOCK_BITS // 2

def split_block(block):
    mask = (1 << HALF_BITS) - 1
    L = (block >> HALF_BITS) & mask
    R = block & mask
    return L, R

def join_block(L, R):
    return ((L & ((1<<HALF_BITS)-1)) << HALF_BITS) | (R & ((1<<HALF_BITS)-1))

def round_function(r_half, round_key):
    # simple non-linear-ish round function: mix and rotate
    # keep results within HALF_BITS
    mask = (1 << HALF_BITS) - 1
    x = (r_half + round_key) & mask
    # a tiny bit of diffusion with rotate and XOR
    rot = ((x << 3) | (x >> (HALF_BITS - 3))) & mask
    return x ^ rot

def key_schedule(key, rounds=4):
    # derive `rounds` 8-bit round keys from 16-bit key deterministically
    # split key into two bytes and mix
    k0 = key & 0xFF
    k1 = (key >> 8) & 0xFF
    keys = []
    for i in range(rounds):
        # simple byte-mixing schedule (toy only)
        keys.append(((k0 * (i+1)) ^ (k1 * (i+2)) ^ i) & 0xFF)
    return keys

def feistel_encrypt(block, key, rounds=4):
    L, R = split_block(block)
    round_keys = key_schedule(key, rounds)
    for k in range(rounds):
        f = round_function(R, round_keys[k])
        L, R = R, L ^ f
    return join_block(L, R)

def feistel_decrypt(block, key, rounds=4):
    # same process reversed
    L, R = split_block(block)
    round_keys = key_schedule(key, rounds)
    for k in reversed(range(rounds)):
        f = round_function(L, round_keys[k])
        L, R = R ^ f, L
    return join_block(L, R)

# Choose a random key and plaintext
random.seed(1)
real_key = random.randint(0, 2**16 - 1)
plaintext = random.randint(0, 2**16 - 1)
ciphertext = feistel_encrypt(plaintext, real_key)

print(f"Real key (hex): 0x{real_key:04x}")
print(f"Plaintext  (hex): 0x{plaintext:04x}")
print(f"Ciphertext (hex): 0x{ciphertext:04x}")
print("Starting brute-force search over 2^16 keys...")

# Brute-force search
start = perf_counter()
found = []
for k in range(2**16):
    if feistel_encrypt(plaintext, k) == ciphertext:
        found.append(k)
end = perf_counter()

print(f"Search finished in {(end-start):.3f} seconds.")
print(f"Number of candidate keys matching the pair: {len(found)}")
# If there are multiple candidates, we might need more plaintext/ciphertext pairs to disambiguate.
for fk in found[:10]:
    dec = feistel_decrypt(ciphertext, fk)
    print(f" Candidate key 0x{fk:04x} -> decrypts ciphertext to 0x{dec:04x}")

# Verify that real_key is among found keys and unique
if real_key in found:
    print("\nSuccess: real key recovered by brute force.")
else:
    print("\nFailure: real key not found (unexpected).")

# If multiple candidates were found, show how adding a second known plaintext disambiguates
if len(found) > 1:
    print("\nDisambiguating with a second known plaintext/ciphertext pair...")
    plaintext2 = (plaintext ^ 0xBEEF) & 0xFFFF
    ciphertext2 = feistel_encrypt(plaintext2, real_key)
    remaining = []
    for fk in found:
        if feistel_encrypt(plaintext2, fk) == ciphertext2:
            remaining.append(fk)
    print(f"Remaining candidates after checking second pair: {len(remaining)}")
    for fk in remaining:
        print(f"  0x{fk:04x}")
else:
    print("No need for disambiguation; single pair sufficed.")


Real key (hex): 0x44cb
Plaintext  (hex): 0x204f
Ciphertext (hex): 0xba5e
Starting brute-force search over 2^16 keys...
Search finished in 0.068 seconds.
Number of candidate keys matching the pair: 6
 Candidate key 0x44cb -> decrypts ciphertext to 0x204f
 Candidate key 0x45cc -> decrypts ciphertext to 0x204f
 Candidate key 0x660f -> decrypts ciphertext to 0x204f
 Candidate key 0xddb9 -> decrypts ciphertext to 0x204f
 Candidate key 0xeef0 -> decrypts ciphertext to 0x204f
 Candidate key 0xfae8 -> decrypts ciphertext to 0x204f

Success: real key recovered by brute force.

Disambiguating with a second known plaintext/ciphertext pair...
Remaining candidates after checking second pair: 1
  0x44cb
